In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)       # 모든 열 출력
pd.set_option('display.width', None)             # 자동 너비 제한 해제
pd.set_option('display.expand_frame_repr', False) # 한 줄로 출력

In [20]:
# GCS 파일 경로
accounts_blockrecord = "gs://codeit-project/votes/accounts_blockrecord.parquet" # 차단 기록 테이블

# GCP 키 경로
key_path = "config/key.json"

# 데이터 불러오기
accounts_blockrecord = pd.read_parquet(accounts_blockrecord, storage_options={"token": key_path})

# 확인
accounts_blockrecord.head()

,id,reason,created_at,block_user_id,user_id
0,1,그냥...,2023-05-04 23:01:53,867483,878476
1,2,친구 사이가 어색해짐,2023-05-05 01:17:08,867190,867564
2,3,나랑 관련 없는 질문을 자꾸 보냄,2023-05-05 01:50:55,875110,875261
3,6,그냥...,2023-05-05 05:21:52,883696,883511
4,7,그냥...,2023-05-05 06:40:34,871349,870177


In [21]:
display(accounts_blockrecord.info())
print('--------')
display(accounts_blockrecord.created_at.min())
print('--------')
display(accounts_blockrecord.created_at.max())
print('--------')
display(accounts_blockrecord.reason.nunique())
print('--------')
display(accounts_blockrecord.reason.value_counts())
print('--------')
display(accounts_blockrecord.user_id.nunique())
print('--------')
display(accounts_blockrecord.block_user_id.nunique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19482 entries, 0 to 19481
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             19482 non-null  int64         
 1   reason         19482 non-null  object        
 2   created_at     19482 non-null  datetime64[ns]
 3   block_user_id  19482 non-null  int64         
 4   user_id        19482 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 761.1+ KB


None

--------


Timestamp('2023-05-04 23:01:53')

--------


...

reason
모르는 사람임               9640
친구 사이가 어색해짐           5805
사칭 계정                 2022
나랑 관련 없는 질문을 자꾸 보냄    1083
너무 많은 양의 질문을 보냄        919
기타                       7
그냥...                    6
Name: count, dtype: int64

--------


12552

--------


16240

In [22]:
# accounts_timelinereport 유저 신고기록 테이블
accounts_timelinereport = "gs://codeit-project/votes/accounts_timelinereport.parquet" # 차단 기록 테이블

# GCP 키 경로
key_path = "config/key.json"

# 데이터 불러오기
accounts_timelinereport = pd.read_parquet(accounts_timelinereport, storage_options={"token": key_path})

# 확인
accounts_timelinereport.head()

,id,reason,created_at,reported_user_id,user_id,user_question_record_id
0,28,타인을 사칭함,2023-05-06 04:44:57,874587,885082,3920588
1,37,친구를 비하하거나 조롱하는 어투,2023-05-06 05:41:19,881048,881298,4018679
2,46,친구를 비하하거나 조롱하는 어투,2023-05-06 06:26:59,887882,881945,4120914
3,64,허위 사실 언급,2023-05-06 07:07:46,888610,893684,4143049
4,65,허위 사실 언급,2023-05-06 07:07:52,888610,893684,4143049


In [23]:
display(accounts_timelinereport.info())
print('-----')
display(accounts_timelinereport.created_at.min())
print('-----')
display(accounts_timelinereport.created_at.max())
print('-----')
display(accounts_timelinereport.reason.nunique())
print('-----')
display(accounts_timelinereport.reason.unique())
print('-----')
display(accounts_timelinereport.reason.value_counts())
print('-----')
display(accounts_timelinereport.user_id.nunique())
print('-----')
display(accounts_timelinereport.reported_user_id.nunique())
print('-----')
display(accounts_timelinereport.user_question_record_id.nunique())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       208 non-null    int64         
 1   reason                   208 non-null    object        
 2   created_at               208 non-null    datetime64[ns]
 3   reported_user_id         208 non-null    int64         
 4   user_id                  208 non-null    int64         
 5   user_question_record_id  208 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.9+ KB


None

-----


Timestamp('2023-05-06 04:44:57')

-----


...

115

-----


125

-----


163